# Config

In [ ]:
import os
import types
from typing import Union

### PDF <> PNG

In [ ]:
%%capture
################
## PDF <> PNG ##
################

# pymupdf (PDF2PNG)
!pip install pymupdf
import fitz
from PIL import Image

# ? (PNG2PDF)
# TODO: find tool

### PDF/PNG > TXT

In [ ]:
%%capture
###################
## PDF/PNG > TXT ##
###################

# pytesseract
!sudo apt install tesseract-ocr
!pip install Pillow==4.0.0
!pip install PIL
!pip install image
!pip install pillow
!pip install imgaug
!pip install pytesseract
!pip install pytorch
%reload_ext autoreload
%autoreload
import pytesseract
import shutil
import os
import random
try:
 from PIL import Image
except ImportError:
 import Image

# PyPDF4
!pip install PyPDF4
import PyPDF4

# pdfminer.six
!pip install pdfminer.six
import pdfminer
from pdfminer.high_level import extract_text

# PyPDF4 txt + info
!pip install PyPDF4
import PyPDF4
from PyPDF4 import PdfFileReader

# TIKA
!pip install tika
from tika import parser

# pdftotext (install failed)
!pip install pdftotext

# Load Files

In [ ]:
# for colab
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

In [ ]:
data_path = os.path.join('/content', 'drive', 'MyDrive', '6_work', 'PHIM', 'data')

input-path = os.path.join(data_path, 'input')
output_path = os.path.join(data_path, 'output')

#pdf_path = os.path.join(input-pat, 'pdf')
#img_path = os.path.join(input-pat, 'img')

# Converter Classes

In [ ]:
class DocForConversion:
    def __init__(self, input_filepath: str,
                 output_filepath: Union[str, None],
                 conversion_tool: str):
        self.input_filepath = input_filepath
        self.input_extension = os.path.splitext(self.input_filepath)
        
        self.output_filepath = None
        self.output_exension = os.path.splitext(self.output_filepath)
        
        self.available_functions = {str(f):f for f in globals().values() if type(f) == types.FunctionType}
        self.selected_tool = self.available_functions[conversion_tool]

    def convert(self):
        try:
            self.selected_tool(self.input_filepath, self.output_filepath)
        except KeyError:
            raise ValueError('Invalid tool selected, please enter the name of one of the following functions:%s' % self.available_functions)

    



# Conversion

e.g. GROBID https://github.com/kermitt2/grobid_client_python

# Evaluation